In [ ]:
import glob #what strings and reads file paths
import os   # operating system
import re   # regex
import pandas as pd 
from scipy import linalg
import numpy as np 
import seaborn as sns #visual heat plots and clutering map
import itertools
from scipy import stats
import operator
import math
import scipy
import matplotlib.pyplot as plt
from collections import defaultdict 
d = defaultdict(list)

In [ ]:
exp = input("Please copy and paste the file path for the experiment data up to and including 'Raw Data'. For example:\n\n ...\OM_PLATFORM_E005\RawData")

In [ ]:
rootdir = r"" + str(exp)
print(rootdir)
file_paths = []
for subdir,dirs, files in os.walk(rootdir):
    for file in files:
        file_paths.append(os.path.join(subdir, file))

print("These are all the file paths that match that experiment number:\n")
file_paths

In [ ]:
#so that they can paste in specific type of file thingy
obs_type = input("What type of observations are you looking for? Ex: 'Objects_Population - Nuclei NonBorder'. Copy the name of that file and paste it into the command prompt below\n")
file_path_match = []
for file_path in file_paths:
    x = re.search(obs_type, file_path)
    if x:
        file_path_match.append(file_path)

In [ ]:
print("These are all the file paths that will lead you to " + obs_type + " in the experiment. There should be as many files as plates.")
file_path_match # file_path_match will be important variable rep the # of plates throughout this code

In [ ]:
# each raw data / plate will become an element in the main_df_list. main_df_list[0] is the df for plate 1 
main_df_list = []
for path in file_path_match:
    main_df = pd.read_csv(path, delimiter = '\t', header = 8, low_memory = False)
    main_df = main_df.drop(['Concentration'], axis = 1) 
    main_df = main_df.dropna(axis =1, how = 'all')
    main_df['location'] = (main_df['Row'].astype(str)+ ', ' + main_df['Column'].astype(str))
    main_df['Compound_Usage'] = ""
    main_df = main_df.dropna(axis = 0) #drop cell if NaN in obs
    main_df = main_df[main_df.columns.drop(list(main_df.filter(regex='Bounding Box')))]
    main_df_list.append(main_df)

In [ ]:
# #main_df_lsit[i] corresponds to a plate
main_col = [col for col in main_df_list[0].columns]
for i in range(len(file_path_match)):
    for name in main_col:
        if name in main_col == "Bounding Box":
            main_df_list.drop("Bounding Box", axis = 1)         
main_df_list[0]

In [ ]:
info = input("Please copy and paste your experiment information file path up to and including INFO. For example:\n\n ...\OM_PLATFORM_E005\Info")
day = input("What day was your experiment imaged? Please enter same date as it exists on the information name")
rootdir = r"" + str(info)
file_paths_layout = []
for subdir,dirs, files in os.walk(rootdir):
    for file in files:
        file_paths_layout.append(os.path.join(subdir, file))
file_paths_layout
file_path_match_layout = []
for file_path in file_paths_layout:
    x = re.search(day, file_path)
    if x and (file_path.endswith('xlsx')) and not (file_path.endswith('plan.xlsx')):
        file_path_match_layout.append(file_path)
print("Below you should see the file name/path associated with the Info for your experiment:\n")
file_path_match_layout

In [ ]:
# should only have 1 layout file
for path in file_path_match_layout:
    layout_df = pd.read_excel(path, sheet_name = 'Tabular')
layout_df

In [ ]:
# this could still use some work since diff cell types were introduced a few days before end
# at the moment this will read from a self-made 'cell_line_layout' document in info
cell_types = input("Do you have different cell types on your plate(s) that you would like to be shown in your clustermap?\n\nPress 1 for YES and 2 for NO\n")
if cell_types == 1:
    rootdir = r"C:\Users\jrozenblit\Box\Programs\NSP900 - Imaging\900-Biology\invitro\experiments\OM_PLATFORM_E" + str(exp_numb)+"\Info"
    cell_line_layout = []
    for subdir,dirs, files in os.walk(rootdir):
        for file in files:
            cell_line_layout.append(os.path.join(subdir, file))
    cell_line_layout
    plan = input("plan\n")
    cell_line_layout_match = []
    for file_path in cell_line_layout:
        x = re.search(plan, file_path)
        if x:
            cell_line_layout_match.append(file_path)
    cell_line_layout_match
if cell_types != 1:
    print("You have indicated that you would not like to have or do not have various cell types marked on your clustermap.")